In [93]:
# 导入相关的库
import os
import numpy as np

# 读取正常邮件的文件名列表
normalList = os.listdir('item5-ss-data/normal/')

# 读取垃圾邮件的文件名列表
spamList = os.listdir("item5-ss-data/spam/")

stopList = []
for line in open('item5-ss-data/stopwords.txt', encoding='utf-8'):
    stopList.append(line[:len(line) - 1])

In [94]:
from jieba import cut
from re import sub
allwordList = [] # allwordList用来存放所有邮件的内容
# 定义一个getwords函数，用于获取邮件的内容
def getwords(path): # 参数path是邮件的完整路径
    wordList = []
    for line in open(path, encoding="utf-8"):
        line = line.strip()
        line = sub(r'[.，。/0-9【】、]','', line)
        line = cut(line)
        # 过滤长度小于1的词
        line = filter(lambda x:len(x) > 1, line)
        wordList.extend(line)
    words = []
    for word in wordList:  # 过滤停用词
        if word not in stopList and word != None: # 该词不在停用词列表里或者不是None值就添加到words列表中
            words.append(word)
    return words

for file in normalList:
    normalListFile = ('item5-ss-data/normal/' + file)
    words = getwords(normalListFile)
    allwordList.append(words)
for file in spamList:
    spamListFile = ('item5-ss-data/spam/' + file)
    words = getwords(spamListFile)
    allwordList.append(words)
    

In [95]:
from collections import Counter
from itertools import chain

# 获取邮件中出现频次最高的十个词
frep = Counter(chain(*allwordList))  # 获取每个词出现的次数
topTen = frep.most_common(10)  # most_common返回的是字典，对应的是出现次数最高的十个词和次数
topWord = []
for i in topTen:
    topWord.append(i[0]) # 将出现次数最高的十个词添加到topWords列表中
vector = []
for word in allwordList:
    temp = list(map(lambda x:word.count(x), topWord)) # 获取高频词在每封邮件出现的次数
    vector.append(temp)
vector = np.array(vector)

In [96]:

# 1表示正常邮件，0表示垃圾邮件
y = np.array([1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0])
x = vector
from sklearn.naive_bayes import MultinomialNB # 导入多项式朴素贝叶斯
# 创建模型并训练
model = MultinomialNB()
model.fit(x, y)

MultinomialNB()

In [97]:
# 测试邮件的内容处理
testList = os.listdir('item5-ss-data/test/')
for testFile in testList: # 读取测试邮件的文件名列表
    testListFile = ('item5-ss-data/test/' + testFile)  # 用来存放测试邮件完整的路径
    words = getwords(testListFile)
    temp = list(map(lambda x:words.count(x),topWord))
    test_x = np.array(temp)
    result = model.predict([test_x])
    if result == 1:
        print(testFile,'是正常邮件')
    else:
        print(testFile,'是垃圾邮件')

normal-test.txt 是正常邮件
spam-test.txt 是垃圾邮件
